In [100]:
import pandas as pd

from transformers import pipeline
import numpy as np
from tqdm import tqdm

In [101]:
movies = pd.read_csv("movies_cleaned.csv")

In [102]:
movies

,id,title,genres,original_language,popularity,release_date,runtime,tagline,vote_average,poster_url,overview,id_description,id_description_keywords_genres
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,8763.998,2023-08-02,116.0,Back for seconds.,7.079,https://image.tmdb.org/t/p/w500/4m1Au3YkjqsxF8...,An exploratory dive into the deepest depths of...,615656 An exploratory dive into the deepest de...,615656 An exploratory dive into the deepest de...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,5953.227,2023-04-05,103.0,Inspired by the actual files of Father Gabriel...,7.433,https://image.tmdb.org/t/p/w500/9JBEPLTPSm0d1m...,Father Gabriele Amorth Chief Exorcist of the V...,758323 Father Gabriele Amorth Chief Exorcist o...,758323 Father Gabriele Amorth Chief Exorcist o...
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,5410.496,2024-07-24,128.0,Come together.,7.765,https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUT...,A listless Wade Wilson toils away in civilian ...,533535 A listless Wade Wilson toils away in ci...,533535 A listless Wade Wilson toils away in ci...
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,5409.104,2023-06-06,127.0,Unite or fall.,7.340,https://image.tmdb.org/t/p/w500/gPbM0MK8CP8A17...,When a new threat capable of destroying the en...,667538 When a new threat capable of destroying...,667538 When a new threat capable of destroying...
4,693134,Dune: Part Two,Science Fiction-Adventure,en,4742.163,2024-02-27,167.0,Long live the fighters.,8.300,https://image.tmdb.org/t/p/w500/czembW0Rk1Ke7l...,Follow the mythic journey of Paul Atreides as ...,693134 Follow the mythic journey of Paul Atrei...,693134 Follow the mythic journey of Paul Atrei...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,562,Die Hard,Action-Thriller,en,43.934,1988-07-15,131.0,Twelve terrorists. One cop. The odds are again...,7.760,https://image.tmdb.org/t/p/w500/yFihWxQcmqcaBR...,NYPD cop John McClane's plan to reconcile with...,562 NYPD cop John McClane's plan to reconcile ...,562 NYPD cop John McClane's plan to reconcile ...
1496,567604,Once Upon a Deadpool,Comedy-Action-Adventure,en,43.913,2018-12-11,118.0,Yule believe in miracles.,7.200,https://image.tmdb.org/t/p/w500/5Ka49BWWyKMXr9...,A kidnapped Fred Savage is forced to endure De...,567604 A kidnapped Fred Savage is forced to en...,567604 A kidnapped Fred Savage is forced to en...
1497,610253,Halloween Kills,Horror-Thriller,en,43.898,2021-10-14,105.0,Evil dies tonight.,6.560,https://image.tmdb.org/t/p/w500/ir9eyz1mtgsohj...,The nightmare isn't over as unstoppable killer...,610253 The nightmare isn't over as unstoppable...,610253 The nightmare isn't over as unstoppable...
1498,277216,Straight Outta Compton,Drama-Music-History,en,43.856,2015-08-11,147.0,The Story of N.W.A.,7.773,https://image.tmdb.org/t/p/w500/9B63hMwU6iICtN...,In 1987 five young men using brutally honest r...,277216 In 1987 five young men using brutally h...,277216 In 1987 five young men using brutally h...


In [104]:
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = 0)

Device set to use cpu


In [105]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
ids = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [106]:
for i in tqdm(range(len(movies))):
    ids.append(movies["id"][i])
    sentences = movies["overview"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 1500/1500 [02:20<00:00, 10.65it/s]


In [107]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["id"] = ids

In [108]:
movies = pd.merge(movies, emotions_df, on = "id")

In [109]:
movies = movies.drop_duplicates(subset=[col for col in movies.columns if col != "genre_list"])

In [110]:
movies

,id,title,genres,original_language,popularity,release_date,runtime,tagline,vote_average,poster_url,overview,id_description,id_description_keywords_genres,anger,disgust,fear,joy,sadness,surprise,neutral
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,8763.998,2023-08-02,116.0,Back for seconds.,7.079,https://image.tmdb.org/t/p/w500/4m1Au3YkjqsxF8...,An exploratory dive into the deepest depths of...,615656 An exploratory dive into the deepest de...,615656 An exploratory dive into the deepest de...,0.064134,0.104007,0.962155,0.040564,0.549477,0.111690,0.078766
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,5953.227,2023-04-05,103.0,Inspired by the actual files of Father Gabriel...,7.433,https://image.tmdb.org/t/p/w500/9JBEPLTPSm0d1m...,Father Gabriele Amorth Chief Exorcist of the V...,758323 Father Gabriele Amorth Chief Exorcist o...,758323 Father Gabriele Amorth Chief Exorcist o...,0.064134,0.104007,0.986530,0.040564,0.549477,0.111690,0.078766
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,5410.496,2024-07-24,128.0,Come together.,7.765,https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUT...,A listless Wade Wilson toils away in civilian ...,533535 A listless Wade Wilson toils away in ci...,533535 A listless Wade Wilson toils away in ci...,0.064134,0.104007,0.877062,0.040564,0.549477,0.872306,0.078766
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,5409.104,2023-06-06,127.0,Unite or fall.,7.340,https://image.tmdb.org/t/p/w500/gPbM0MK8CP8A17...,When a new threat capable of destroying the en...,667538 When a new threat capable of destroying...,667538 When a new threat capable of destroying...,0.088041,0.104007,0.874339,0.040564,0.820426,0.111690,0.078766
4,693134,Dune: Part Two,Science Fiction-Adventure,en,4742.163,2024-02-27,167.0,Long live the fighters.,8.300,https://image.tmdb.org/t/p/w500/czembW0Rk1Ke7l...,Follow the mythic journey of Paul Atreides as ...,693134 Follow the mythic journey of Paul Atrei...,693134 Follow the mythic journey of Paul Atrei...,0.898895,0.104007,0.971447,0.040564,0.549477,0.111690,0.078766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,562,Die Hard,Action-Thriller,en,43.934,1988-07-15,131.0,Twelve terrorists. One cop. The odds are again...,7.760,https://image.tmdb.org/t/p/w500/yFihWxQcmqcaBR...,NYPD cop John McClane's plan to reconcile with...,562 NYPD cop John McClane's plan to reconcile ...,562 NYPD cop John McClane's plan to reconcile ...,0.265528,0.104007,0.768946,0.040564,0.549477,0.111690,0.078766
1548,567604,Once Upon a Deadpool,Comedy-Action-Adventure,en,43.913,2018-12-11,118.0,Yule believe in miracles.,7.200,https://image.tmdb.org/t/p/w500/5Ka49BWWyKMXr9...,A kidnapped Fred Savage is forced to endure De...,567604 A kidnapped Fred Savage is forced to en...,567604 A kidnapped Fred Savage is forced to en...,0.064134,0.104007,0.051363,0.040564,0.549477,0.839842,0.078766
1549,610253,Halloween Kills,Horror-Thriller,en,43.898,2021-10-14,105.0,Evil dies tonight.,6.560,https://image.tmdb.org/t/p/w500/ir9eyz1mtgsohj...,The nightmare isn't over as unstoppable killer...,610253 The nightmare isn't over as unstoppable...,610253 The nightmare isn't over as unstoppable...,0.117060,0.104007,0.947423,0.040564,0.549477,0.664027,0.078766
1550,277216,Straight Outta Compton,Drama-Music-History,en,43.856,2015-08-11,147.0,The Story of N.W.A.,7.773,https://image.tmdb.org/t/p/w500/9B63hMwU6iICtN...,In 1987 five young men using brutally honest r...,277216 In 1987 five young men using brutally h...,277216 In 1987 five young men using brutally h...,0.961849,0.111984,0.763324,0.046140,0.549477,0.270279,0.092808


In [111]:
movies["genre_list"] = movies["genres"].fillna("").apply(lambda x: x.split("-"))

In [112]:
all_genres = [genre for sublist in movies["genre_list"] for genre in sublist]
unique_genres = sorted(set(all_genres))

In [113]:
unique_genres

['Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western']

In [114]:
movies

,id,title,genres,original_language,popularity,release_date,runtime,tagline,vote_average,poster_url,...,id_description,id_description_keywords_genres,anger,disgust,fear,joy,sadness,surprise,neutral,genre_list
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,8763.998,2023-08-02,116.0,Back for seconds.,7.079,https://image.tmdb.org/t/p/w500/4m1Au3YkjqsxF8...,...,615656 An exploratory dive into the deepest de...,615656 An exploratory dive into the deepest de...,0.064134,0.104007,0.962155,0.040564,0.549477,0.111690,0.078766,"[Action, Science Fiction, Horror]"
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,5953.227,2023-04-05,103.0,Inspired by the actual files of Father Gabriel...,7.433,https://image.tmdb.org/t/p/w500/9JBEPLTPSm0d1m...,...,758323 Father Gabriele Amorth Chief Exorcist o...,758323 Father Gabriele Amorth Chief Exorcist o...,0.064134,0.104007,0.986530,0.040564,0.549477,0.111690,0.078766,"[Horror, Mystery, Thriller]"
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,5410.496,2024-07-24,128.0,Come together.,7.765,https://image.tmdb.org/t/p/w500/8cdWjvZQUExUUT...,...,533535 A listless Wade Wilson toils away in ci...,533535 A listless Wade Wilson toils away in ci...,0.064134,0.104007,0.877062,0.040564,0.549477,0.872306,0.078766,"[Action, Comedy, Science Fiction]"
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,5409.104,2023-06-06,127.0,Unite or fall.,7.340,https://image.tmdb.org/t/p/w500/gPbM0MK8CP8A17...,...,667538 When a new threat capable of destroying...,667538 When a new threat capable of destroying...,0.088041,0.104007,0.874339,0.040564,0.820426,0.111690,0.078766,"[Action, Adventure, Science Fiction]"
4,693134,Dune: Part Two,Science Fiction-Adventure,en,4742.163,2024-02-27,167.0,Long live the fighters.,8.300,https://image.tmdb.org/t/p/w500/czembW0Rk1Ke7l...,...,693134 Follow the mythic journey of Paul Atrei...,693134 Follow the mythic journey of Paul Atrei...,0.898895,0.104007,0.971447,0.040564,0.549477,0.111690,0.078766,"[Science Fiction, Adventure]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,562,Die Hard,Action-Thriller,en,43.934,1988-07-15,131.0,Twelve terrorists. One cop. The odds are again...,7.760,https://image.tmdb.org/t/p/w500/yFihWxQcmqcaBR...,...,562 NYPD cop John McClane's plan to reconcile ...,562 NYPD cop John McClane's plan to reconcile ...,0.265528,0.104007,0.768946,0.040564,0.549477,0.111690,0.078766,"[Action, Thriller]"
1548,567604,Once Upon a Deadpool,Comedy-Action-Adventure,en,43.913,2018-12-11,118.0,Yule believe in miracles.,7.200,https://image.tmdb.org/t/p/w500/5Ka49BWWyKMXr9...,...,567604 A kidnapped Fred Savage is forced to en...,567604 A kidnapped Fred Savage is forced to en...,0.064134,0.104007,0.051363,0.040564,0.549477,0.839842,0.078766,"[Comedy, Action, Adventure]"
1549,610253,Halloween Kills,Horror-Thriller,en,43.898,2021-10-14,105.0,Evil dies tonight.,6.560,https://image.tmdb.org/t/p/w500/ir9eyz1mtgsohj...,...,610253 The nightmare isn't over as unstoppable...,610253 The nightmare isn't over as unstoppable...,0.117060,0.104007,0.947423,0.040564,0.549477,0.664027,0.078766,"[Horror, Thriller]"
1550,277216,Straight Outta Compton,Drama-Music-History,en,43.856,2015-08-11,147.0,The Story of N.W.A.,7.773,https://image.tmdb.org/t/p/w500/9B63hMwU6iICtN...,...,277216 In 1987 five young men using brutally h...,277216 In 1987 five young men using brutally h...,0.961849,0.111984,0.763324,0.046140,0.549477,0.270279,0.092808,"[Drama, Music, History]"


In [115]:
movies.to_csv("movies_with_emotions.csv", index = False)